In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1,
              0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[1, :])

In [5]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [7]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [8]:
W = eval_model(X_st, y, iterations=1000, alpha=1e-4)

100 [ 0.49277624 -0.15016005  0.64709669  1.5172415 ] 1.2309273637360083
200 [ 0.48885913 -0.1620201   0.64649828  1.51148191] 1.211972737521506
300 [ 0.48496327 -0.17384257  0.64589355  1.50575204] 1.1931571091358864
400 [ 0.48108916 -0.18562535  0.6452828   1.50005293] 1.1744864101901549
500 [ 0.4772373  -0.19736619  0.64466637  1.49438568] 1.155967045243567
600 [ 0.47340821 -0.20906258  0.64404467  1.48875145] 1.1376059352778145
700 [ 0.46960243 -0.22071181  0.64341817  1.48315148] 1.119410563104108
800 [ 0.46582056 -0.2323109   0.64278738  1.4775871 ] 1.101389020053552
900 [ 0.46206317 -0.24385659  0.64215292  1.47205973] 1.0835500530355255
1000 [ 0.45833091 -0.2553453   0.64151548  1.46657088] 1.0659031107284278


## 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.  

In [9]:
def calc_logloss(y, y_pred):
    predicted = y_pred.copy()
    for i in range(len(predicted)):
        if predicted[i] == 0.0:
            predicted[i] = 0.01
        elif predicted[i] == 1.0:
            predicted[i] = 0.99
      
    err = - np.mean(y * np.log(predicted) + (1.0 - y) * np.log(1.0 - predicted))
    err = np.sum(err)
    return err

## 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [10]:
def eval_model_(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    #if i % (iterations / 10) == 0:
      #print(i, W, alpha, err)
    return W, err, alpha, iterations

In [11]:
best_params = None
err_min = 1000
i = 0
for iterations in [4000, 5000, 3500, 2000, 3000]:
    for alpha in [1e-3, 1e-8, 1e-2, 0.005, 1e-5, 1e-4]:
        params = W, err, alpha, iterations = eval_model_(X_st, y, iterations=iterations, alpha=alpha)
        if err < err_min:
            err_min = err
            best_params = params

print(f'The best parameters: iterations {params[-1]}, alpha {params[-2]}')

The best parameters: iterations 3000, alpha 0.0001


## 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [12]:
W = eval_model(X_st, y, iterations=3000, alpha=1e-4)

300 [ 0.48496327 -0.17384257  0.64589355  1.50575204] 1.1931571091358864
600 [ 0.47340821 -0.20906258  0.64404467  1.48875145] 1.1376059352778145
900 [ 0.46206317 -0.24385659  0.64215292  1.47205973] 1.0835500530355255
1200 [ 0.45094443 -0.27813585  0.64023486  1.45571533] 1.031226861538826
1500 [ 0.44007076 -0.31178459  0.63831445  1.43976514] 0.9809336291045673
1800 [ 0.42946372 -0.34465401  0.63642475  1.42426596] 0.9330340710859316
2100 [ 0.41914781 -0.37655851  0.63460935  1.40928523] 0.8879543554013178
2400 [ 0.40914981 -0.40727726  0.63292223  1.39489992] 0.8461598024239766
2700 [ 0.39949726 -0.43656504  0.63142538  1.38119251] 0.808105289969183
3000 [ 0.39021583 -0.46417456  0.63018332  1.36824355] 0.7741616988816424


In [40]:
def calc_pred_proba(X, W):
    z = np.dot(W, X)
    y_pred_proba = np.floor(sigmoid(z))
    return y_pred_proba

In [41]:
calc_pred_proba(X, W)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [37]:
def calc_pred(W, X):
    m = X.shape[1]
    y_pred = np.zeros((1, m), dtype = np.int64)
    z = W.T @ X
    y_pred_proba = sigmoid(z)
    
    for i in range(len(y_pred_proba)):
        if (y_pred_proba[i] > 0.5): 
            y_pred[:, i] = 1
        elif (y_pred_proba[i] <= 0.5):
            y_pred[:, i] = 0
    return y_pred

In [38]:
y_pred = calc_pred(W, X)
y_pred

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

## 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

#### Accuracy

In [22]:
def accuracy_(y_pred, y):
    return len(y_pred[y == y_pred]) / len(y_pred)

In [23]:
accuracy_(y_pred, y)

5.0

#### Correlation Matrix

In [24]:
def correlation_matrix(y_pred, y):
    TP = len(y_pred[(y == y_pred) & (y_pred == 1)])
    FN = len(y_pred[(y != y_pred) & (y_pred == 0)])
    FP = len(y_pred[(y != y_pred) & (y_pred == 1)])
    TN = len(y_pred[(y == y_pred) & (y_pred == 0)])
    corr_matrix = pd.DataFrame.from_dict(
        {'y_pred': ['a(x) = +1', 'a(x) = -1'], 
         'y = +1': [TP, FN], 
         'y = -1': [FP, TN]}).set_index('y_pred')
    return corr_matrix

In [25]:
correlation_matrix(y_pred, y)

,y = +1,y = -1
y_pred,,
a(x) = +1,5,5
a(x) = -1,0,0


#### Precision

In [26]:
def precision_(y_pred, y):
    return len(y_pred[(y == y_pred) & (y_pred == 1)]) / (
        len(y_pred[(y == y_pred) & (y_pred == 1)]) + len(y_pred[(y != y_pred) & (y_pred == 1)]))

In [27]:
precision_(y_pred, y)

0.5

#### Recall

In [28]:
def recall_(y_pred, y):
    return len(y_pred[(y == y_pred) & (y_pred == 1)]) / (
        len(y_pred[(y == y_pred) & (y_pred == 1)]) + len(y_pred[(y != y_pred) & (y_pred == 0)]))

In [29]:
recall_(y_pred, y)

1.0

#### F1-score

In [30]:
def f1_(y_pred, y):
    return 2 * precision_(y_pred, y) * recall_(y_pred, y) / (precision_(y_pred, y) + recall_(y_pred, y))

In [31]:
f1_(y_pred, y)

0.6666666666666666

## 6. Могла ли модель переобучиться? Почему?

Да. Данных мало. Нет разбиения на тестовую и валидационную выборки.

## 7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

#### L1

In [32]:
def eval_model_L1(X, y, iterations, alpha=1e-5, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T)  + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [33]:
eval_model_L1(X, y, iterations=400, alpha=1e-5, lambda_=1e-7)

40 [ 0.49651415 -0.1387843   0.46368854  1.52274986] 2.3076102609207965
80 [ 0.49631415 -0.1393043   0.27968854  1.52246986] 2.3076102609207965
120 [ 0.49611415 -0.1398243   0.09568854  1.52218986] 2.3076102609207965
160 [ 0.49599515 -0.1401124  -0.00414988  1.52205088] 0.8479506859997198
200 [ 0.49597158 -0.14012408 -0.00414987  1.52207923] 0.8479439030016106
240 [ 0.49594801 -0.14013575 -0.00414985  1.52210757] 0.8479371202613354
280 [ 0.49592443 -0.14014743 -0.00414983  1.52213592] 0.8479303377785896
320 [ 0.49590086 -0.1401591  -0.00414982  1.52216427] 0.847923555553351
360 [ 0.49587729 -0.14017077 -0.0041498   1.52219261] 0.8479167735855981
400 [ 0.49585371 -0.14018244 -0.00414978  1.52222096] 0.847909991875308


array([ 0.49585371, -0.14018244, -0.00414978,  1.52222096])

#### L2

In [34]:
def eval_model_L2(X, y, iterations, alpha=1e-5, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [35]:
eval_model_L2(X, y, iterations=300, alpha=1e-5, lambda_=1e-8)

30 [ 0.49656415 -0.1386543   0.50968854  1.52281986] 2.3076102609207965
60 [ 0.49641415 -0.1390443   0.37168854  1.52260986] 2.3076102609207965
90 [ 0.49626415 -0.1394343   0.23368854  1.52239986] 2.3076102609207965
120 [ 0.49611415 -0.1398243   0.09568854  1.52218986] 2.3076102609207965
150 [ 0.49600105 -0.14010949 -0.00414989  1.52204379] 0.8479527685868111
180 [ 0.49598337 -0.14011824 -0.00414988  1.52206505] 0.8479472941023294
210 [ 0.49596569 -0.140127   -0.00414986  1.52208631] 0.8479422069127409
240 [ 0.49594801 -0.14013575 -0.00414985  1.52210757] 0.847937119868023
270 [ 0.49593033 -0.14014451 -0.00414984  1.52212883] 0.8479320329681664
300 [ 0.49591265 -0.14015326 -0.00414983  1.52215009] 0.8479269462131617


array([ 0.49591265, -0.14015326, -0.00414983,  1.52215009])